In [1]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

/usr/local/lib/python3.7/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
# load training and testing set
# define shuffle data loader?
# got csv files from https://pjreddie.com/projects/mnist-in-csv/
HOME="/home/adsgray"
DIR=HOME + "/code/mnist"
TRAINING=DIR + "/mnist_train.csv"
TESTING=DIR + "/mnist_test.csv"



In [204]:
trainingfile = open(TRAINING, "r")
testingfile = open(TESTING, "r")
traininglines = trainingfile.readlines()
testinglines = testingfile.readlines()

def tensor_from_string(s):
    l = [int(i) for i in s.split(",")]
    #map(int, l)
    return tensor(l)

    
training_raw = torch.stack([tensor_from_string(s) for s in traininglines])
testing_raw = torch.stack([tensor_from_string(s) for s in testinglines])



In [391]:

training_labels = training_raw[:,0]
training_images = training_raw[:,1:].float()/255

testing_labels = testing_raw[:,0]
testing_images = testing_raw[:,1:].float()/255
#show_image(t[0].view(28,28))
#testing_labels[59]
#show_image(testing_images[59].view(28,28))

testing_images.shape[0]

10000

In [250]:
# functions to select a random sample
import random

def random_indexes(num, max):
    return [random.randrange(0,max) for i in range(num)]

def training_sample(num):
    max = training_images.shape[0]
    idxs = random_indexes(num, max)
    return [(training_images[idxs], training_labels[idxs])]



In [449]:
# define cost function

from torch import exp
def softmax(x): return exp(x) / exp(x).sum(dim=1, keepdim=True)
def sigmoid(x): return 1/(1+torch.exp(-x))

fudge=1.

def mnist_loss(predictions, targets):
    return F.nll_loss(F.log_softmax(predictions, dim=1), targets)

    
def mnist_loss2(predictions, targets):
    lp = torch.log(predictions)
    sm = softmax(lp)
    #print("sm: ", sm)
    predictions = -sm
    #predictions = softmax(predictions)
    return predictions[range(targets.shape[0]), targets].mean()
   

In [446]:
# initialize parameters
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()


w1 = init_params((28*28,30))
b1 = init_params(30)
w2 = init_params((30,10))
b2 = init_params(10)

params = (w1,b1,w2,b2)


In [447]:
# define simple network
def simple_net(xb): 
    res = xb@w1 + b1
    # relu
    res = res.max(tensor(0.0))
    res = res@w2 + b2
    return res

In [451]:

print_acc = True

def accuracy(preds, y):
    prednums = preds[range(y.shape[0]),y]
    maxes = torch.max(preds, dim=1)
    cor = prednums == maxes.values
    acc = cor.float().mean()
    return acc
    


def calc_grad(xb, yb, model):
    preds = model(xb)
    acc = accuracy(preds, yb)
    if print_acc: 
        print(acc)
    #print("preds: ", preds)
    #print("yb: ", yb)
    loss = mnist_loss(preds, yb)
    print("loss: ", loss)
    loss.backward()
    
    
def train_epoch(size, model, lr, params):
    dl = training_sample(size)
    for xb,yb in dl:
        calc_grad(xb, yb, model)
        for p in params:
            #print("grad: ", p.grad)
            p.data -= p.grad*lr
            p.grad.zero_()
            
    


In [553]:
# training loop
#https://stackoverflow.com/questions/60032073/select-specific-rows-of-2d-pytorch-tensor


#def mse(preds, targets): return ((preds-targets)**2).mean().sqrt()
#def linear1(xb): return xb@weights + bias


def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb,yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)

lr = 0.01
num=10
size=2000
# train
def train_model(size, model, params, lr, num):
    for i in range(num):
        train_epoch(size, model, lr, params)
       


#print(params)
train_model(size, simple_net, params, lr, num)







tensor(0.7345)
loss:  tensor(1.0070, grad_fn=<NllLossBackward>)
tensor(0.7315)
loss:  tensor(1.0919, grad_fn=<NllLossBackward>)
tensor(0.7490)
loss:  tensor(0.9846, grad_fn=<NllLossBackward>)
tensor(0.7400)
loss:  tensor(0.9229, grad_fn=<NllLossBackward>)
tensor(0.7410)
loss:  tensor(1.0358, grad_fn=<NllLossBackward>)
tensor(0.7285)
loss:  tensor(0.9771, grad_fn=<NllLossBackward>)
tensor(0.7600)
loss:  tensor(0.9357, grad_fn=<NllLossBackward>)
tensor(0.7475)
loss:  tensor(1.0096, grad_fn=<NllLossBackward>)
tensor(0.7375)
loss:  tensor(1.0324, grad_fn=<NllLossBackward>)
tensor(0.7150)
loss:  tensor(1.1140, grad_fn=<NllLossBackward>)


In [554]:
# write out model to disk
import pickle

def save_model():
    model = {'w1':w1, 'b1': b1, 'w2':w2, 'b2': b2}
    with open('mymodel.pkl', 'wb') as handle:
        pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

def read_model():
    with open('filename.pickle', 'rb') as handle:
        model = pickle.load(handle)
    w1 = model['w1']
    b1 = model['b1']
    w2 = model['w2']
    b2 = model['b2']


    
save_model()


